# Gradient descent

Due to some {ref}`computational issues <computational-issues>` it could be inpractical to calculate $\widehat{\boldsymbol w}$ by the formula {eq}`lin-reg-solution`. If so, a numerical optimization method can be used. The most common one is **gradient descent**.

```{prf:algorithm} Gradient descent
:label: GD
:nonumber:

To solve the optimization problem

$$
    \mathcal L(\boldsymbol w) \to \min\limits_{\boldsymbol w}
$$

do the followind steps:

1. initialize $\boldsymbol w$ by some random values (e.g., from $\mathcal N(0, 1$))
2. choose **tolerance** $\varepsilon > 0$ and **learning rate** $\eta > 0$
3. while $\Vert \nabla\mathcal L(\boldsymbol w) \Vert > \varepsilon$ do the **gradient step**

    $$
    \boldsymbol w := \boldsymbol w - \eta\nabla\mathcal L(\boldsymbol w)
    $$
4. return $\boldsymbol w$
```

```{note}
If condition $\Vert \nabla\mathcal L(\boldsymbol w) \Vert > \varepsilon$ holds for too long, the loop in step 3 terminates after some number iterations `max_iter`.
```

For ordinary linear regression the loss function is proportional to 

$$
    \mathcal L(\boldsymbol w) = \frac 12\Vert\boldsymbol {Xw} - \boldsymbol y \Vert_2^2,
$$

therefore, $\nabla \mathcal L(\boldsymbol w) = \boldsymbol X^\top(\boldsymbol{Xw} - \boldsymbol y)$.

Now let's implement this algorithm:

In [98]:
import numpy as np

def linear_regression_gd(X, y, learning_rate=0.01, tol=1e-3, max_iter=10000):
    w = np.random.normal(size=X.shape[1])
    gradient = X.T.dot(X.dot(w) - y)
    for i in range(max_iter):
        if np.linalg.norm(gradient) <= tol:
            return w
        w -= learning_rate * gradient
        gradient = X.T.dot(X.dot(w) - y)
    print("max_iter exceeded")
    return w

Try in on some synthetic data:

In [99]:
X = np.random.normal(size=(50, 20))
y = np.random.rand(50)
w = linear_regression_gd(X, y, tol=1e-4)
print("MSE:", np.mean((X.dot(w) - y)**2))

MSE: 0.1830720788779873


In [100]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression(fit_intercept=False)
LR.fit(X, y)
print("r-score:", LR.score(X, y))
print("MSE:", np.mean((LR.predict(X) - y) ** 2))
print("Difference from w:", np.linalg.norm(LR.coef_ - w))

r-score: -1.032649581837315
MSE: 0.18307207885813592
Difference from w: 1.0782324101636377e-05


Now apply to Boston dataset:

In [101]:
import pandas as pd
boston = pd.read_csv("../ISLP_datsets/Boston.csv").drop("Unnamed: 0", axis=1)
target = boston['medv']
train = boston.drop(['medv'], axis=1)
X_train = np.hstack([np.ones(506)[:, None], train])

In [111]:
weights = linear_regression_gd(X_train, target.values, learning_rate=9e-9)
print("MSE:", np.mean((X_train.dot(weights) - target)**2))

max_iter exceeded
MSE: 46.63558173285131


Looks not so good...